In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Coffee")
bs = 5
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = bs


Loading the Coffee dataset...
The dataset Coffee was loaded.


In [3]:
dtcr_model = DTCRModel(config)
dtcr_model

DTCRModel(
  (encoder): BidirectionalDRNN(
    (_regular_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 100, batch_first=True)
        (1): GRU(100, 50, batch_first=True)
        (2): GRU(50, 50, batch_first=True)
      )
    )
    (_backwards_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 100, batch_first=True)
        (1): GRU(100, 50, batch_first=True)
        (2): GRU(50, 50, batch_first=True)
      )
    )
  )
  (decoder): DTCRDecoder(
    (_rnn): GRU(400, 400, batch_first=True)
    (_linear): Linear(in_features=400, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [4]:

recons_criterion = config.decoding_criterion()
classify_criterion = config.classifier_criterion()
optimizer = config.optimizer(dtcr_model.parameters(),
                             eps=config.learning_rate)
for epoch in range(1000):
    print("Starting epoch {}:".format(epoch + 1))
    dtcr_model.train_step(train_dl, test_dl, recons_criterion, classify_criterion,
                          optimizer)

Starting epoch 1:
[5] loss: 1.690093493461609, classify: 0.6933191895484925, recons: 0.9967743039131165
Starting epoch 2:
[5] loss: 1.6885727882385253, classify: 0.693252444267273, recons: 0.9953203678131104
Starting epoch 3:
[5] loss: 1.6871797084808349, classify: 0.6932074666023255, recons: 0.9939722418785095
Starting epoch 4:
[5] loss: 1.685389471054077, classify: 0.6931790113449097, recons: 0.9922104716300965
Starting epoch 5:
[5] loss: 1.6829765081405639, classify: 0.6931627631187439, recons: 0.9898137331008912
Starting epoch 6:
[5] loss: 1.6777445077896118, classify: 0.6931540131568908, recons: 0.984590494632721
Starting epoch 7:
[5] loss: 1.6579204559326173, classify: 0.6931498527526856, recons: 0.9647706031799317
Starting epoch 8:
[5] loss: 1.6178243637084961, classify: 0.6931481003761292, recons: 0.924676239490509
Starting epoch 9:
[5] loss: 1.5741846323013307, classify: 0.6931474924087524, recons: 0.8810371518135071
Starting epoch 10:
[5] loss: 1.611794114112854, classify: 0.

KeyboardInterrupt: 

In [ ]:
for name, param in dtcr_model.named_parameters():
    print(name, param.grad.norm())

encoder._regular_drnn._layers.0.weight_ih_l0 tensor(0.0151)
encoder._regular_drnn._layers.0.weight_hh_l0 tensor(0.0045)
encoder._regular_drnn._layers.0.bias_ih_l0 tensor(0.0073)
encoder._regular_drnn._layers.0.bias_hh_l0 tensor(0.0036)
encoder._regular_drnn._layers.1.weight_ih_l0 tensor(0.0051)
encoder._regular_drnn._layers.1.weight_hh_l0 tensor(0.0017)
encoder._regular_drnn._layers.1.bias_ih_l0 tensor(0.0046)
encoder._regular_drnn._layers.1.bias_hh_l0 tensor(0.0023)
encoder._regular_drnn._layers.2.weight_ih_l0 tensor(0.0021)
encoder._regular_drnn._layers.2.weight_hh_l0 tensor(0.0009)
encoder._regular_drnn._layers.2.bias_ih_l0 tensor(0.0028)
encoder._regular_drnn._layers.2.bias_hh_l0 tensor(0.0014)
encoder._backwards_drnn._layers.0.weight_ih_l0 tensor(0.0109)
encoder._backwards_drnn._layers.0.weight_hh_l0 tensor(0.0029)
encoder._backwards_drnn._layers.0.bias_ih_l0 tensor(0.0052)
encoder._backwards_drnn._layers.0.bias_hh_l0 tensor(0.0026)
encoder._backwards_drnn._layers.1.weight_ih_l0 t